In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
working_dir = '/content/gdrive/MyDrive/Object_detection'

import os
os.makedirs(working_dir, exist_ok=True)

%cd {working_dir}

/content/gdrive/MyDrive/Object_detection


In [3]:
# !git clone https://github.com/ultralytics/yolov5  # clone

%cd yolov5
%pip install -r requirements.txt  # install

# !git clone https://github.com/nwojke/deep_sort.git
# !wget https://github.com/anushkadhiman/ObjectTracking-DeepSORT-YOLOv3-TF2/raw/master/model_data/coco/mars-small128.pb

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-163-g016e046 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 24.1/78.2 GB disk)


In [4]:
!python detect.py --weights yolov5s.pt --img 640 --conf 0.25 --source /content/gdrive/MyDrive/Object_detection/yolov5/data/videos/detection.mp4

detect: weights=['yolov5s.pt'], source=/content/gdrive/MyDrive/Object_detection/yolov5/data/videos/detection.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
requirements: /content/gdrive/MyDrive/Object_detection/requirements.txt not found, check failed.
YOLOv5 🚀 v7.0-163-g016e046 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
video 1/1 (1/443) /content/gdrive/MyDrive/Object_detection/yolov5/data/videos/detection.mp4: 384x640 18 persons, 3 bicycles, 1 bench, 45.1ms
video 1/1 (2/443) /content/gdrive/MyDrive/Object_detection/yolov5/data/videos/detecti

In [5]:
labels = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck","boat"]

In [6]:
NUM_CLASS = { 0: 'person',
              1: 'bicycle',
              2: 'car',
              3: 'motorbike',
              4: 'aeroplane',
              5: 'bus',
              6: 'train',
              7: 'truck',
              8: 'boat',
              9: 'traffic-light',
              10: 'fire-hydrant',
              11: 'stop-sign',
              12: 'parking-meter',
              13: 'bench',
              14: 'bird',
              15: 'cat',
              16: 'dog',
              17: 'horse',
              18: 'sheep',
              19: 'cow',
              20: 'elephant',
              21: 'bear',
              22: 'zebra',
              23: 'giraffe',
              24: 'backpack',
              25: 'umbrella',
              26: 'handbag',
              27: 'tie',
              28: 'suitcase',
              29: 'frisbee',
              30: 'skis',
              31: 'snowboard',
              32: 'sports-ball',
              33: 'kite',
              34: 'baseball-bat',
              35: 'baseball-glove',
              36: 'skateboard',
              37: 'surfboard',
              38: 'tennis-racket',
              39: 'bottle',
              40: 'wine-glass',
              41: 'cup',
              42: 'fork',
              43: 'knife',
              44: 'spoon',
              45: 'bowl',
              46: 'banana',
              47: 'apple',
              48: 'sandwich',
              49: 'orange',
              50: 'broccoli',
              51: 'carrot',
              52: 'hot-dog',
              53: 'pizza',
              54: 'donut',
              55: 'cake',
              56: 'chair',
              57: 'sofa',
              58: 'pottedplant',
              59: 'bed',
              60: 'diningtable',
              61: 'toilet',
              62: 'tvmonitor',
              63: 'laptop',
              64: 'mouse',
              65: 'remote',
              66: 'keyboard',
              67: 'cell-phone',
              68: 'microwave',
              69: 'oven',
              70: 'toaster',
              71: 'sink',
              72: 'refrigerator',
              73: 'book',
              74: 'clock',
              75: 'vase',
              76: 'scissors',
              77: 'teddy-bear',
              78: 'hair-drier',
              79: 'toothbrush'
}

In [7]:
%%writefile detect_trackings.py
"""
Run YOLOv5 detection inference on images, videos, directories, globs, YouTube, webcam, streams, etc.

Usage - sources:
    $ python detect.py --weights yolov5s.pt --source 0                               # webcam
                                                     img.jpg                         # image
                                                     vid.mp4                         # video
                                                     screen                          # screenshot
                                                     path/                           # directory
                                                     list.txt                        # list of images
                                                     list.streams                    # list of streams
                                                     'path/*.jpg'                    # glob
                                                     'https://youtu.be/Zgi9g1ksQHc'  # YouTube
                                                     'rtsp://example.com/media.mp4'  # RTSP, RTMP, HTTP stream

Usage - formats:
    $ python detect.py --weights yolov5s.pt                 # PyTorch
                                 yolov5s.torchscript        # TorchScript
                                 yolov5s.onnx               # ONNX Runtime or OpenCV DNN with --dnn
                                 yolov5s_openvino_model     # OpenVINO
                                 yolov5s.engine             # TensorRT
                                 yolov5s.mlmodel            # CoreML (macOS-only)
                                 yolov5s_saved_model        # TensorFlow SavedModel
                                 yolov5s.pb                 # TensorFlow GraphDef
                                 yolov5s.tflite             # TensorFlow Lite
                                 yolov5s_edgetpu.tflite     # TensorFlow Edge TPU
                                 yolov5s_paddle_model       # PaddlePaddle
"""

import argparse
import os
import platform
import sys
from pathlib import Path

import torch

FILE = Path(__file__).resolve()
ROOT = FILE.parents[0]  # YOLOv5 root directory
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))  # add ROOT to PATH
ROOT = Path(os.path.relpath(ROOT, Path.cwd()))  # relative

# DeepSORT -> Importing DeepSORT.
from deep_sort.application_util import preprocessing
from deep_sort.deep_sort import nn_matching
from deep_sort.deep_sort.detection import Detection
from deep_sort.deep_sort.tracker import Tracker
from deep_sort.tools import generate_detections as gdet

from models.common import DetectMultiBackend
from utils.dataloaders import IMG_FORMATS, VID_FORMATS, LoadImages, LoadScreenshots, LoadStreams
from utils.general import (LOGGER, Profile, check_file, check_img_size, check_imshow, check_requirements, colorstr, cv2,
                           increment_path, non_max_suppression, print_args, scale_boxes, strip_optimizer, xyxy2xywh)
from utils.plots import Annotator, colors, save_one_box
from utils.torch_utils import select_device, smart_inference_mode


@smart_inference_mode()
def object_tracking_detection(weights=ROOT / 'yolov5s.pt',  # model path or triton URL
                              source=ROOT / 'data/images',  # file/dir/URL/glob/screen/0(webcam)
                              data=ROOT / 'data/coco128.yaml',  # dataset.yaml path
                              imgsz=(640, 640),  # inference size (height, width)
                              conf_thres=0.25,  # confidence threshold
                              iou_thres=0.45,  # NMS IOU threshold
                              max_det=1000,  # maximum detections per image
                              device='',  # cuda device, i.e. 0 or 0,1,2,3 or cpu
                              view_img=False,  # show results
                              save_txt=False,  # save results to *.txt
                              save_conf=False,  # save confidences in --save-txt labels
                              save_crop=False,  # save cropped prediction boxes
                              nosave=False,  # do not save images/videos
                              classes=None,  # filter by class: --class 0, or --class 0 2 3
                              agnostic_nms=False,  # class-agnostic NMS
                              augment=False,  # augmented inference
                              visualize=False,  # visualize features
                              update=False,  # update all models
                              project=ROOT / 'runs/detect',  # save results to project/name
                              name='exp',  # save results to project/name
                              exist_ok=False,  # existing project/name ok, do not increment
                              line_thickness=3,  # bounding box thickness (pixels)
                              hide_labels=False,  # hide labels
                              hide_conf=False,  # hide confidences
                              half=False,  # use FP16 half-precision inference
                              dnn=False,  # use OpenCV DNN for ONNX inference
                              vid_stride=1,  # video frame-rate stride
    ):
    source = str(source)
    save_img = not nosave and not source.endswith('.txt')  # save inference images
    is_file = Path(source).suffix[1:] in (IMG_FORMATS + VID_FORMATS)
    is_url = source.lower().startswith(('rtsp://', 'rtmp://', 'http://', 'https://'))
    webcam = source.isnumeric() or source.endswith('.streams') or (is_url and not is_file)
    screenshot = source.lower().startswith('screen')
    if is_url and is_file:
        source = check_file(source)  # download

    # DeepSORT -> Initializing tracker.
    max_cosine_distance = 0.4
    nn_budget = None
    model_filename = 'mars-small128.pb'
    encoder = gdet.create_box_encoder(model_filename, batch_size=1)
    metric = nn_matching.NearestNeighborDistanceMetric("cosine", max_cosine_distance, nn_budget)
    tracker = Tracker(metric)

    # Directories
    save_dir = increment_path(Path(project) / name, exist_ok=exist_ok)  # increment run
    (save_dir / 'labels' if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  # make dir

    # Load model
    device = select_device(device)
    model = DetectMultiBackend(weights, device=device, dnn=dnn, data=data, fp16=half)
    stride, names, pt = model.stride, model.names, model.pt
    imgsz = check_img_size(imgsz, s=stride)  # check image size

    # Dataloader
    bs = 1  # batch_size
    if webcam:
        view_img = check_imshow(warn=True)
        dataset = LoadStreams(source, img_size=imgsz, stride=stride, auto=pt, vid_stride=vid_stride)
        bs = len(dataset)
    elif screenshot:
        dataset = LoadScreenshots(source, img_size=imgsz, stride=stride, auto=pt)
    else:
        dataset = LoadImages(source, img_size=imgsz, stride=stride, auto=pt, vid_stride=vid_stride)
    vid_path, vid_writer = [None] * bs, [None] * bs

    # Run inference
    model.warmup(imgsz=(1 if pt or model.triton else bs, 3, *imgsz))  # warmup
    seen, windows, dt = 0, [], (Profile(), Profile(), Profile())

    frame_idx=0

    for path, im, im0s, vid_cap, s in dataset:
        with dt[0]:
            im = torch.from_numpy(im).to(model.device)
            im = im.half() if model.fp16 else im.float()  # uint8 to fp16/32
            im /= 255  # 0 - 255 to 0.0 - 1.0
            if len(im.shape) == 3:
                im = im[None]  # expand for batch dim

        # Inference
        with dt[1]:
            visualize = increment_path(save_dir / Path(path).stem, mkdir=True) if visualize else False
            pred = model(im, augment=augment, visualize=visualize)

        # NMS
        with dt[2]:
            pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)

        # Second-stage classifier (optional)
        # pred = utils.general.apply_classifier(pred, classifier_model, im, im0s)

        frame_idx=frame_idx+1

        # Process predictions
        for i, det in enumerate(pred):  # per image
            seen += 1
            if webcam:  # batch_size >= 1
                p, im0, frame = path[i], im0s[i].copy(), dataset.count
                s += f'{i}: '
            else:
                p, im0, frame = path, im0s.copy(), getattr(dataset, 'frame', 0)

            p = Path(p)  # to Path
            save_path = str(save_dir / p.name)  # im.jpg
            txt_path = str(save_dir / 'labels' / p.stem) + ('' if dataset.mode == 'image' else f'_{frame}')  # im.txt
            s += '%gx%g ' % im.shape[2:]  # print string
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
            imc = im0.copy() if save_crop else im0  # for save_crop
            annotator = Annotator(im0, line_width=line_thickness, example=str(names))

            if len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_boxes(im.shape[2:], det[:, :4], im0.shape).round()

                # Print results
                for c in det[:, 5].unique():
                    n = (det[:, 5] == c).sum()  # detections per class
                    s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string

                # Write results for object detection
                # for *xyxy, conf, cls in reversed(det):
                #     if save_txt:  # Write to file
                #         xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                #         line = (cls, *xywh, conf) if save_conf else (cls, *xywh)  # label format
                #         with open(f'{txt_path}.txt', 'a') as f:
                #             f.write(('%g ' * len(line)).rstrip() % line + '\n')

                #     if save_img or save_crop or view_img:  # Add bbox to image
                #         c = int(cls)  # integer class
                #         label = None if hide_labels else (names[c] if hide_conf else f'{names[c]} {conf:.2f}')
                #         annotator.box_label(xyxy, label, color=colors(c, True))
                #     if save_crop:
                #         save_one_box(xyxy, imc, file=save_dir / 'crops' / names[c] / f'{p.stem}.jpg', BGR=True)

                # DeepSORT -> Extracting Bounding boxes and its confidence scores.
                bboxes = []
                scores = []
                det_names = []
                label = ""
                for *boxes, conf, cls in det:
                    bbox_left = min([boxes[0].item(), boxes[2].item()])
                    bbox_top = min([boxes[1].item(), boxes[3].item()])
                    bbox_w = abs(boxes[0].item() - boxes[2].item())
                    bbox_h = abs(boxes[1].item() - boxes[3].item())
                    box = [bbox_left, bbox_top, bbox_w, bbox_h]
                    bboxes.append(box)
                    scores.append(conf.item())
                    c = int(cls)  # integer class
                    det_names.append(names[c])
                    label = None if hide_labels else (names[c] if hide_conf else f'{names[c]} {conf:.2f}')

                # DeepSORT -> Getting appearance features of the object.
                features = encoder(im0, bboxes)
                # DeepSORT -> Storing all the required info in a list.
                detections = [Detection(bbox, score, class_name, feature) for bbox, score, class_name, feature in zip(bboxes, scores, det_names, features)]

                # DeepSORT -> Predicting Tracks.
                tracker.predict()
                tracker.update(detections)
                #track_time = time.time() - prev_time

                # DeepSORT -> Plotting the tracks.
                for track in tracker.tracks:
                    if not track.is_confirmed() or track.time_since_update > 1:
                        continue

                    # DeepSORT -> Changing track bbox to top left, bottom right coordinates.
                    bbox = list(track.to_tlbr())

                    # DeepSORT -> Writing Track bounding box and ID on the frame using OpenCV.
                    class_name = track.get_class()
                    txt = class_name + ':' + str(track.track_id)
                    (label_width,label_height), baseline = cv2.getTextSize(txt , cv2.FONT_HERSHEY_SIMPLEX,1,1)
                    top_left = tuple(map(int,[int(bbox[0]),int(bbox[1])-(label_height+baseline)]))
                    top_right = tuple(map(int,[int(bbox[0])+label_width,int(bbox[1])]))
                    org = tuple(map(int,[int(bbox[0]),int(bbox[1])-baseline]))

                    cv2.rectangle(im0, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (255,0,0), 1)
                    cv2.rectangle(im0, top_left, top_right, (255,0,0), -1)
                    cv2.putText(im0, txt, org, cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 1)

                    # DeepSORT -> Saving Track predictions into a text file.

                    if save_txt:  # Write to file
                        save_format = '{frame},{id},{x1},{y1},{w},{h},{x},{y},{z}\n'
                        print("txt: ", txt_path, '.txt')
                        line = save_format.format(frame=frame_idx, id=track.track_id, x1=int(bbox[0]), y1=int(bbox[1]), w=int(bbox[2]- bbox[0]), h=int(bbox[3]-bbox[1]), x = -1, y = -1, z = -1)
                        with open(f'{txt_path}.txt', 'a') as f:
                            f.write(('%g ' * len(line)).rstrip() % line + '\n')

            # Stream results
            im0 = annotator.result()
            if view_img:
                if platform.system() == 'Linux' and p not in windows:
                    windows.append(p)
                    cv2.namedWindow(str(p), cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO)  # allow window resize (Linux)
                    cv2.resizeWindow(str(p), im0.shape[1], im0.shape[0])
                cv2.imshow(str(p), im0)
                cv2.waitKey(1)  # 1 millisecond

            # Save results (image with detections)
            if save_img:
                if dataset.mode == 'image':
                    cv2.imwrite(save_path, im0)
                else:  # 'video' or 'stream'
                    if vid_path[i] != save_path:  # new video
                        vid_path[i] = save_path
                        if isinstance(vid_writer[i], cv2.VideoWriter):
                            vid_writer[i].release()  # release previous video writer
                        if vid_cap:  # video
                            fps = vid_cap.get(cv2.CAP_PROP_FPS)
                            w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                            h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                        else:  # stream
                            fps, w, h = 30, im0.shape[1], im0.shape[0]
                        save_path = str(Path(save_path).with_suffix('.mp4'))  # force *.mp4 suffix on results videos
                        vid_writer[i] = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
                    vid_writer[i].write(im0)

        # Print time (inference-only)
        LOGGER.info(f"{s}{'' if len(det) else '(no detections), '}{dt[1].dt * 1E3:.1f}ms")

    # Print results
    t = tuple(x.t / seen * 1E3 for x in dt)  # speeds per image
    LOGGER.info(f'Speed: %.1fms pre-process, %.1fms inference, %.1fms NMS per image at shape {(1, 3, *imgsz)}' % t)
    if save_txt or save_img:
        s = f"\n{len(list(save_dir.glob('labels/*.txt')))} labels saved to {save_dir / 'labels'}" if save_txt else ''
        LOGGER.info(f"Results saved to {colorstr('bold', save_dir)}{s}")
    if update:
        strip_optimizer(weights[0])  # update model (to fix SourceChangeWarning)


def parse_opt():
    parser = argparse.ArgumentParser()
    parser.add_argument('--weights', nargs='+', type=str, default=ROOT / 'yolov5s.pt', help='model path or triton URL')
    parser.add_argument('--source', type=str, default=ROOT / 'data/images', help='file/dir/URL/glob/screen/0(webcam)')
    parser.add_argument('--data', type=str, default=ROOT / 'data/coco128.yaml', help='(optional) dataset.yaml path')
    parser.add_argument('--imgsz', '--img', '--img-size', nargs='+', type=int, default=[640], help='inference size h,w')
    parser.add_argument('--conf-thres', type=float, default=0.25, help='confidence threshold')
    parser.add_argument('--iou-thres', type=float, default=0.45, help='NMS IoU threshold')
    parser.add_argument('--max-det', type=int, default=1000, help='maximum detections per image')
    parser.add_argument('--device', default='', help='cuda device, i.e. 0 or 0,1,2,3 or cpu')
    parser.add_argument('--view-img', action='store_true', help='show results')
    parser.add_argument('--save-txt', action='store_true', help='save results to *.txt')
    parser.add_argument('--save-conf', action='store_true', help='save confidences in --save-txt labels')
    parser.add_argument('--save-crop', action='store_true', help='save cropped prediction boxes')
    parser.add_argument('--nosave', action='store_true', help='do not save images/videos')
    parser.add_argument('--classes', nargs='+', type=int, help='filter by class: --classes 0, or --classes 0 2 3')
    parser.add_argument('--agnostic-nms', action='store_true', help='class-agnostic NMS')
    parser.add_argument('--augment', action='store_true', help='augmented inference')
    parser.add_argument('--visualize', action='store_true', help='visualize features')
    parser.add_argument('--update', action='store_true', help='update all models')
    parser.add_argument('--project', default=ROOT / 'runs/detect', help='save results to project/name')
    parser.add_argument('--name', default='exp', help='save results to project/name')
    parser.add_argument('--exist-ok', action='store_true', help='existing project/name ok, do not increment')
    parser.add_argument('--line-thickness', default=3, type=int, help='bounding box thickness (pixels)')
    parser.add_argument('--hide-labels', default=False, action='store_true', help='hide labels')
    parser.add_argument('--hide-conf', default=False, action='store_true', help='hide confidences')
    parser.add_argument('--half', action='store_true', help='use FP16 half-precision inference')
    parser.add_argument('--dnn', action='store_true', help='use OpenCV DNN for ONNX inference')
    parser.add_argument('--vid-stride', type=int, default=1, help='video frame-rate stride')
    opt = parser.parse_args()
    opt.imgsz *= 2 if len(opt.imgsz) == 1 else 1  # expand
    print_args(vars(opt))
    return opt


def main(opt):
    check_requirements(exclude=('tensorboard', 'thop'))
    object_tracking_detection(**vars(opt))


if __name__ == '__main__':
    opt = parse_opt()
    main(opt)

Writing detect_trackings.py


In [8]:
!python detect_trackings.py --weights yolov5s.pt --img 640 --conf 0.25 --class 0 1 2 --source /content/gdrive/MyDrive/Object_detection/yolov5/data/videos/detection.mp4

detect_trackings: weights=['yolov5s.pt'], source=/content/gdrive/MyDrive/Object_detection/yolov5/data/videos/detection.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=[0, 1, 2], agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
requirements: /content/gdrive/MyDrive/Object_detection/requirements.txt not found, check failed.
YOLOv5 🚀 v7.0-163-g016e046 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
video 1/1 (1/443) /content/gdrive/MyDrive/Object_detection/yolov5/data/videos/detection.mp4: 384x640 18 persons, 3 bicycles, 43.4ms
video 1/1 (2/443) /content/gdrive/MyDrive/Object_detection/yolov5/data/videos/d

In [9]:
!python train.py --img 640 --batch 16 --epochs 10 --data dataset.yaml --weights yolov5s.pt --cache

train: weights=yolov5s.pt, cfg=, data=dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
remote: Enumerating objects: 326, done.
remote: Counting objects: 100% (269/269), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 326 (delta 214), reused 227 (delta 193), pack-reused 57
Receiving objects: 100% (326/326), 173.87 KiB | 983.00 KiB/s, done.
Resolving deltas: 100% (238/238), completed with 42 local objects.
Command 'git fetch origin' timed out after 5 seconds
requirements: /con

In [10]:
!python detect.py --weights /content/gdrive/MyDrive/Object_detection/yolov5/runs/train/exp/weights/best.pt --img 640 --conf 0.25 --source /content/gdrive/MyDrive/Object_detection/yolov5/data/videos/test_video.mp4

detect: weights=['/content/gdrive/MyDrive/Object_detection/yolov5/runs/train/exp/weights/best.pt'], source=/content/gdrive/MyDrive/Object_detection/yolov5/data/videos/test_video.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
requirements: /content/gdrive/MyDrive/Object_detection/requirements.txt not found, check failed.
YOLOv5 🚀 v7.0-163-g016e046 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7023610 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/38) /content/gdrive/MyDrive/Object_detection/yolov5/data/videos/test_video.mp4: 384x640 2 Cars, 44.2ms
video 1/1 (2/38) /content/gdri